In [1]:
import os
import os.path
import sys

import pandas as pd
import numpy as np
from importlib import reload

In [43]:
LOCAL_PATH = 'data'
TRACE_NAME = 'youtube'
RAW_TRACE = '{trace_name}.pcapng'.format(trace_name=TRACE_NAME)
TCP_TRACE_FEATURE_FILE = '{trace_name}_tcp_pkt.csv'.format(trace_name=TRACE_NAME)
UDP_TRACE_FEATURE_FILE = '{trace_name}_udp_pkt.csv'.format(trace_name=TRACE_NAME)
TCP_FLOW_FEATURE_FILE = '{trace_name}_tcp_flow.csv'.format(trace_name=TRACE_NAME)
UDP_FLOW_FEATURE_FILE = '{trace_name}_udp_flow.csv'.format(trace_name=TRACE_NAME)
# BUCKET_NAME = '' # replace with your bucket name
KEY = '' # replace with your object key

In [44]:
if not os.path.exists(os.path.join(LOCAL_PATH, RAW_TRACE)):
    if not os.path.exists(LOCAL_PATH):
        os.mkdir(LOCAL_PATH)
    
    import boto3
    import botocore

    s3 = boto3.resource('s3')

    try:
        s3.Bucket(BUCKET_NAME).download_file(KEY, os.path.join(LOCAL_PATH, RAW_TRACE))
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise

In [45]:
# convert raw trace to readable udp and tcp packet feature csv file
from python import packet_feature
reload(packet_feature)
%time packet_feature.udp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) , os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE), is_cluster=True)
%time packet_feature.tcp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) , os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE), is_cluster=True)

Packet feature file already exists.
CPU times: user 8.86 ms, sys: 0 ns, total: 8.86 ms
Wall time: 25.6 ms
Packet feature file already exists.
CPU times: user 1.8 s, sys: 32.2 ms, total: 1.83 s
Wall time: 1.88 s


In [46]:
def to_csv(trace_df, extract_func, file, features, max_byte_per_flow):
    df = extract_func(trace_df, 1.0, upsampled=True, max_byte_per_flow=max_byte_per_flow)
    if df.shape[0] == 0:
        pd.DataFrame(columns=features).to_csv(file, index=False)
    else:
        df[features].to_csv(file, index=False)

In [47]:
from python import flow_feature
reload(flow_feature)

FEATURES = ['avg(pkt_len)', 'stddev(pkt_len)', 'fb_ratio', 'inter_arrival_time', 'pkt_count', 'duration', 'tot_pkt_len', 'fwd_pkt_len', 'bwd_pkt_len']
MAX_BYTE_PER_FLOW = 1 * 1024 * 1024 # 1MB

tcp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))
udp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
for trace_df in [tcp_trace_df, udp_trace_df]:
    if tcp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.tcp_generate, os.path.join(LOCAL_PATH, TCP_FLOW_FEATURE_FILE), FEATURES, max_byte_per_flow=MAX_BYTE_PER_FLOW)
    elif udp_trace_df is trace_df:
        to_csv(trace_df, flow_feature.udp_generate, os.path.join(LOCAL_PATH, UDP_FLOW_FEATURE_FILE), FEATURES, max_byte_per_flow=MAX_BYTE_PER_FLOW)
    else:
        raise

tcp flow, flow byte limit->1048576: 65980it [00:27, 2438.91it/s]          
/share/apps/python3/3.5.3/intel/lib/python3.5/site-packages/numpy-1.12.0-py3.5-linux-x86_64.egg/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/share/apps/python3/3.5.3/intel/lib/python3.5/site-packages/numpy-1.12.0-py3.5-linux-x86_64.egg/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
udp flow, flow byte limit->1048576: 271it [00:00, 3270.42it/s]           
